**Import Library**

In [ ]:
!pip install tensorflowjs

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflowjs as tfjs

**Load Dataset**

In [ ]:
df = pd.read_csv("/content/Dataset_Stunting.csv")
df = df.dropna()

**Encode Target**

In [ ]:
label_encoder = LabelEncoder()
df["status_gizi_who_encoded"] = label_encoder.fit_transform(df["status_gizi_who"])

**Fitur dan label**

In [ ]:
X = df.drop(["status_gizi_who", "status_gizi_who_encoded"], axis=1)
y = df["status_gizi_who_encoded"]

In [ ]:
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

**Normalisasi**

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

**Model Neural Network**

In [ ]:
model = Sequential([
    Input(shape=(4,)),  # WAJIB agar bisa dibaca tfjs
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

model.compile(
optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

**Train**

In [ ]:
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2)

Epoch 1/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.8366 - loss: 0.3871 - val_accuracy: 0.9016 - val_loss: 0.2119
Epoch 2/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8956 - loss: 0.2316 - val_accuracy: 0.9100 - val_loss: 0.1975
Epoch 3/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9020 - loss: 0.2130 - val_accuracy: 0.9145 - val_loss: 0.1810
Epoch 4/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9049 - loss: 0.2071 - val_accuracy: 0.9118 - val_loss: 0.1912
Epoch 5/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9051 - loss: 0.2030 - val_accuracy: 0.9140 - val_loss: 0.1825
Epoch 6/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9067 - loss: 0.1983 - val_accuracy: 0.9158 - val_loss: 0.1804
Epoch 7/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9072 - loss: 0.1983 - val_accuracy: 0.9215 - val_loss: 0.1706
Epoch 8/40
2418/2418 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9085 - loss: 0.19

**Evaluasi**

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n✅ Akurasi pada data uji: {accuracy:.2f}")

756/756 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9616 - loss: 0.0945

✅ Akurasi pada data uji: 0.96


**Prediksi**

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

756/756 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


**Hasil**

In [ ]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(
    y_test, y_pred, target_names=label_encoder.classes_
))


Confusion Matrix:
 [[17323     3   270]
 [    0  3893   219]
 [  218   258  1991]]

Classification Report:
                   precision    recall  f1-score   support

          normal       0.99      0.98      0.99     17596
severely stunted       0.94      0.95      0.94      4112
         stunted       0.80      0.81      0.80      2467

        accuracy                           0.96     24175
       macro avg       0.91      0.91      0.91     24175
    weighted avg       0.96      0.96      0.96     24175



**SIMPAN MODEL DAN ALAT PRA-PROSES**

In [ ]:
model.save("model_stunting_nn.h5")
joblib.dump(scaler, "scaler_stunting.pkl")
joblib.dump(label_encoder, "label_encoder_stunting.pkl")

['label_encoder_stunting.pkl']

In [ ]:
tfjs.converters.save_keras_model(model, "tfjs_model")
print("\n✅ Model berhasil dikonversi ke TensorFlow.js di folder 'tfjs_model/'")

failed to lookup keras version from the file,
    this is likely a weight only file

✅ Model berhasil dikonversi ke TensorFlow.js di folder 'tfjs_model/'
